# jQAssistant Demo
* Neo4j-Server starten
 * `mvn jqassistant:server`
* Browser öffnen
 * http://localhost:7474/browser/
* jQAssistant Dokumentation: http://buschmais.github.io/jqassistant/doc/1.3.0/#_java_plugin
 

## Beispiel-Queries

### Setup
Cypher Extension für Jupyter laden

In [1]:
%load_ext cypher

### TOP 10 Klasse mit den meisten Methoden auflisten

In [2]:
%%cypher
MATCH 
  (t:Type)-[:DECLARES]->(m:Method)
RETURN t.fqn as Typ, COUNT(m) as Methodenanzahl
ORDER BY Methodenanzahl DESC
LIMIT 10

10 rows affected.


Typ,Methodenanzahl
org.springframework.samples.petclinic.model.Owner,21
org.springframework.samples.petclinic.model.Pet,16
org.springframework.samples.petclinic.web.OwnerControllerTests,13
org.springframework.samples.petclinic.service.AbstractClinicServiceTests,12
org.springframework.samples.petclinic.model.Vet,11
org.springframework.samples.petclinic.service.ClinicServiceImpl,11
org.springframework.samples.petclinic.repository.jdbc.JdbcPet,10
org.springframework.samples.petclinic.model.Visit,10
org.springframework.samples.petclinic.web.OwnerController,9
org.springframework.samples.petclinic.service.ClinicService,9


### Statische, geschriebene Variablen

In [3]:
%%cypher
MATCH (c:Class)-[:DECLARES]->(f:Field)<-[w:WRITES]-(m:Method)
WHERE 
    EXISTS(f.static) AND NOT EXISTS(f.final)
RETURN 
    c.name as InClass, 
    m.name as theMethod, 
    w.lineNumber as writesInLine, 
    f.name as toStaticField

3 rows affected.


InClass,theMethod,writesInLine,toStaticField
OwnerController,processFindForm,112,ownersIndexes
OwnerController,processFindForm,112,ownersIndexes
BrokenSingleton,getInstance,11,INSTANCE


### Zyklischen Abhängigkeiten zwischen Packages

In [4]:
%%cypher
MATCH
    (p1:Package)-[:DEPENDS_ON]->(p2:Package),
    path=shortestPath((p2)-[:DEPENDS_ON*]->(p1))
WHERE
    p1<>p2
RETURN
    p1 AS Package, EXTRACT(p IN nodes(path) | p.fqn) AS Cycle
ORDER BY
    Package.fqn

6 rows affected.


Package,Cycle
"{'fileName': '/org/springframework/samples/petclinic/model', 'fqn': 'org.springframework.samples.petclinic.model', 'name': 'model'}","['org.springframework.samples.petclinic.repository', 'org.springframework.samples.petclinic.model']"
"{'fqn': 'org.springframework.samples.petclinic.repository', 'fileName': '/org/springframework/samples/petclinic/repository', 'name': 'repository'}","['org.springframework.samples.petclinic.util', 'org.springframework.samples.petclinic.model', 'org.springframework.samples.petclinic.repository']"
"{'fqn': 'org.springframework.samples.petclinic.repository', 'fileName': '/org/springframework/samples/petclinic/repository', 'name': 'repository'}","['org.springframework.samples.petclinic.model', 'org.springframework.samples.petclinic.repository']"
"{'fqn': 'org.springframework.samples.petclinic.service', 'fileName': '/org/springframework/samples/petclinic/service', 'name': 'service'}","['org.springframework.samples.petclinic.web', 'org.springframework.samples.petclinic.service']"
"{'fileName': '/org/springframework/samples/petclinic/util', 'fqn': 'org.springframework.samples.petclinic.util', 'name': 'util'}","['org.springframework.samples.petclinic.model', 'org.springframework.samples.petclinic.repository', 'org.springframework.samples.petclinic.util']"
"{'fileName': '/org/springframework/samples/petclinic/web', 'fqn': 'org.springframework.samples.petclinic.web', 'name': 'web'}","['org.springframework.samples.petclinic.service', 'org.springframework.samples.petclinic.web']"


### Aggregation von Werten über fachliche Bereiche

In [5]:
%%cypher
MATCH 
  (t:Type)-[:BELONGS_TO]->(s:Subdomain),
  (t)-[:HAS_CHANGE]->(ch:Change)
RETURN 
  s.name as ASubdomain,
  COUNT(DISTINCT t) as Types,
  COUNT(DISTINCT ch) as Changes
ORDER BY Types DESC

0 rows affected.


ASubdomain,Types,Changes


### Code-Abhängigkeiten zwischen fachlichen Bereiche

In [6]:
%%cypher
MATCH
  (t1:Type)-[:BELONGS_TO]->(s1:Subdomain),
  (t2:Type)-[:BELONGS_TO]->(s2:Subdomain),
  (t1)-[d:DEPENDS_ON]->(t2)
WHERE
  s1 <> s2
WITH
  s1, s2, count(d) as weight
RETURN
  s1.name as Subdomain, collect(s2.name) as Dependencies
ORDER BY
  Subdomain

0 rows affected.


Subdomain,Dependencies
